# Nginx

Nginx is a versatile tool that can be utilized as a web server, reverse proxy, load balancer, and much more. This page provides an overview of the fundamentals of working with Nginx.

**Sources:**

- [Nginx handbook](https://www.freecodecamp.org/news/the-nginx-handbook/).
- [Default variables available in NGINX](https://nginx.org/en/docs/varindex.html).

For examples, we will run nginx in a docker container - the following cell runs nginx in a docker container.

In [14]:
docker run -itd --name experiment_nginx --rm -p 80:80 nginx

7ac76c6e2c9997a545084ead5bba2165e03522530f79bb6ee4ecc921bea39645


**Note** don't forget to stop container after all.

In [17]:
docker stop experiment_nginx

experiment_nginx


## Basics of configuration

Once the container is running, we can access nginx at `localhost:80`. The following cell displays a basic nginx welcome page.

In [2]:
cat << EOF | displayHTML
<iframe 
    srcdoc='$(curl -s localhost:80)' 
    width='100%' 
    height='600px' 
    style='border:none;'
>
</iframe>
EOF

Welcome to nginx! 

 
 
 Welcome to nginx! 
 If you see this page, the nginx web server is successfully installed and
working. Further configuration is required. 

 For online documentation and support please refer to
 nginx.org . 
Commercial support is available at
 nginx.com . 

 Thank you for using nginx. 
 
 ' 
 width='100%' 
 height='600px' 
 style='border:none;'
>

Nginx is mainly controlled by its configuration files, so it's important to understand where they are. They are usually located in the `/etc/nginx/` folder. The following cell shows the contents of this folder.

In [3]:
docker exec experiment_nginx ls -l /etc/nginx/

total 32
drwxr-xr-x 1 root root 4096 Jul 22 07:53 conf.d
-rw-r--r-- 1 root root 1007 May 28 13:22 fastcgi_params
-rw-r--r-- 1 root root 5349 May 28 13:22 mime.types
lrwxrwxrwx 1 root root   22 May 29 16:45 modules -> /usr/lib/nginx/modules
-rw-r--r-- 1 root root  648 May 29 16:45 nginx.conf
-rw-r--r-- 1 root root  636 May 28 13:22 scgi_params
-rw-r--r-- 1 root root  664 May 28 13:22 uwsgi_params


The most important file here is `nginx.conf` - it's the central nginx configuration file. Let's replace the default `nginx.conf` with the new configuration specified in the following cell.

In [4]:
cat << EOF > nginx_files/nginx.conf
events {}

http {
    server {
        listen 80;
        return 200 "this is new message from nginx";
    }
}
EOF

Now replace the file with a new one and use `nginx -s reload` to make nginx use the new configuration.

In [5]:
docker cp $(pwd)/nginx_files/nginx.conf experiment_nginx:/etc/nginx/nginx.conf
docker exec experiment_nginx nginx -s reload

Successfully copied 3.07kB to experiment_nginx:/etc/nginx/nginx.conf
2024/07/22 07:53:49 [notice] 67#67: signal process started


Here is showen how default behaviour of the nginx changed.

In [6]:
curl localhost:80

this is new message from nginx


We just got message specified in the `nginx.conf`.

## Reverse proxy (location)

Another way to use nginx is as a reverse proxy, so you can redirect requests. You can define it with the `proxy_pass` directive in the `server` context.

The following config file makes requests to nginx to redirect to google.com.

In [15]:
cat << EOF > nginx_files/proxy_example.conf
events {}

http {
    server {
        listen 80;
        location / {proxy_pass "https://google.com/";}
    }
}
EOF

Updating configuration and reloading nginx.

In [16]:
docker cp $(pwd)/nginx_files/proxy_example.conf experiment_nginx:/etc/nginx/nginx.conf
docker exec experiment_nginx nginx -s reload

Successfully copied 3.07kB to experiment_nginx:/etc/nginx/nginx.conf
2024/07/22 08:00:17 [notice] 61#61: signal process started


Now, when we access `localhost:80`, we get a message that we've been redirected.

In [20]:
curl -s localhost:80 | displayHTML

301 Moved 
 301 Moved 
The document has moved
 here .

By adding the `-L` option to the `curl` command, we can follow redirects, allowing us to retrieve Google's main page.

In [19]:
curl -sL localhost:80 | displayHTML

,,Пашыраны пошук


## Logs

Logging in nginx is really usefull in process of the trouble shouting. There are two log files in nginx:

- `/var/log/nginx/access.log` - where all attempts to accept are logged.
- `/var/log/nginx/error.log` - where logged cases leading to errors.

For reasons that are still unclear, we need to reorder the log files in order to be able to read them. So in the following cell we do this for our container.

In [10]:
docker exec -i experiment_nginx bash << EOF

rm /var/log/nginx/access.log /var/log/nginx/error.log
touch /var/log/nginx/access.log /var/log/nginx/error.log
nginx -s reload
EOF

And here is a request to nginx to print the contents of `access.log`.

In [11]:
curl localhost:80 &> /dev/null
docker exec experiment_nginx cat /var/log/nginx/access.log

172.17.0.1 - - [22/Jul/2024:07:54:09 +0000] "GET / HTTP/1.1" 301 220 "-" "curl/7.81.0"
